In [1]:
import os
import hw
import env_ingredient_add
def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [2]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [ ]:
# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
context = diagram.CreateDefaultContext()
traj = hw.create_small_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj)

hw.create_small_trajectory(diagram,meshcat)
hw.run_simulation(meshcat,sim,20)

controller = diagram.GetSubsystemByName("station").GetSubsystemByName("controller")

print(traj_bot.controller.get_parameters().get_joint_position_limits())
fix_base_pos(traj_bot.controller.get_parameters(), 3*[True])
print(f"after running: {traj_bot.controller.get_parameters().get_joint_position_limits()}")

# print_diagram(diagram)


In [ ]:
controller = diagram.GetSubsystemByName("station").GetSubsystemByName("mobile_iiwa.controller")

print(controller.get_parameters().get_joint_position_limits())
fix_base_pos(traj_bot.controller.get_parameters(), 3*[True])
print(f"after running: {traj_bot.controller.get_parameters().get_joint_position_limits()}")